## Leah's Strava Stats 2016-2020

Leah started running consistently in the Fall 2016 right around Thanksgiving. At the time she had significant IT Band issues which limited her runs to maybe 3-5 miles at the most.  leah is uncertain about why she is talking about herself in third person.


## API authentication

This strava API changed in the past year. There are three parts to authenticating
The workflow below shows a one time authentication where the token is good for ~6 hours

there is a token refresh option that i will look into next

1) Get authorization code from authorization page. This is a one time, manual step. 

## This works --  key being the activity:read_all scope 
This is important as ti gives you read access to all of your activities-- get code

https://www.strava.com/oauth/authorize?client_id=33627&redirect_uri=http://localhost&response_type=code&scope=activity:read_all

the above call will return a code. use the code in the link below which gives you a token
Returns something like below. grabt the code
http://localhost/?state=&code=cd52fb78fc73dba7b439771d5843d9f96b3f5240&scope=read,activity:read_all

**d52fb78fc73dba7b439771d5843d9f96b3f5240**

2. Next you can grab a token - the url is: 
https://www.strava.com/oauth/token?client_id=YOUR_ID&client_secret=YOUR_SECRET&code=YOUR_CODE&grant_type=authorization_code


In [1]:
# import requests

# from stravalib import Client
# from flask import request

# requests.post("https://www.strava.com/oauth/token?client_id=33627&client_secret=0c073c33c77e81d2a786799ac5d14670eedd74ce&code=a0d8d1ba81bbcd8ed131918f65f836492d9c10f9&grant_type=authorization_code")

# Authentication With Strava 

This is the trickiest part but it's actually straight forward. 
First, you need to allow access to your personal strava data. the link below can be 
created from the uses client_id.

The high level workflow is

1. Authenticate using a URL like the one below that defines the scope (the permissions) of what you can access. `activity:read_all` seems to work well. This step requires the user to login to strava and accept the permissions. 
The return of this acceptance if you don't have an app / website set up is a page that looks like an error - but the url of the page has a `code=code-here-long-string-of-characters` element. Copy that code to use in the next 
step which allows you to get a token.

https://www.strava.com/oauth/authorize?client_id=33627&redirect_uri=http://localhost&response_type=code&scope=activity:read_all

`example code returned:a1cf8bbf0d46072d5d3da9676b3a9b9bda5807c8`


2. Once you have the code you can then use a POST request to get a JSON return that has a few items including a token, a refresh token and other things. Luckily for this step you can actually use the stravalib package to 
access the interface that returns the token. 

If you are using postman to test the link below, you can male a POST request to see the returned json.

https://www.strava.com/oauth/token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_SECRET_API&code=CODE_RETURNED_FROM_FIRST_AUTHENTICATION_STEP&grant_type=authorization_code


TO authenticate with `stravalib`, use the method below.

```python
client.exchange_code_for_token(client_id=MY_STRAVA_CLIENT_ID,
                                               client_secret=MY_STRAVA_CLIENT_SECRET,
                                               code=code)

```

**Important - the code that is returned from the intiial user authentication can only be used once. 
after that it's invalid.**

In [2]:
# to convert -- CLI
# jupyter nbconvert --to hide_code_html strava.ipynb
import os
from datetime import datetime
from glob import glob

import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import seaborn as sns
import folium

from stravalib.client import Client
from stravalib.util import limiter


from process_strava import authenticate
from process_strava import get_activities, get_stream_data 


In [3]:
client = Client(rate_limiter=limiter.DefaultRateLimiter())
client = authenticate("strava-secrets.txt", client)

athlete_info = client.get_athlete()
athlete_info

Token still valid, expires at Sun, 21 Feb 2021 21:48:46 MST


<Athlete id=10295934 firstname='Leah' lastname='Wasser'>

In [ ]:
# # Create plotly plot for cum miles
# # I might. be able to do this better with a for loop and regular vanilly pands plotting (use code below)
# trace0 = go.Scatter(
#     y=cum_miles[2016],
#     mode='lines',
#     name='2016',
#     connectgaps=True,
#     line=dict(
#         color="grey")
# )

# trace1 = go.Scatter(
#     y=cum_miles[2017],
#     mode='lines',
#     name='2017',
#     connectgaps=True,
#     line=dict(
#         color="wheat")
# )
# trace2 = go.Scatter(
#     y=cum_miles[2018],
#     mode='lines',
#     name='2018',
#     connectgaps=True,
#     line=dict(
#         color="lightsalmon")
# )

# trace3 = go.Scatter(
#     y=cum_miles[2019],
#     mode='lines',
#     name='2019',
#     connectgaps=True,
#     line=dict(
#         color="purple")
# )

# trace4 = go.Scatter(
#     y=cum_miles[2020],
#     mode='lines',
#     name='2020',
#     connectgaps=True,
#     line=dict(
#         color="purple")
# )

# layout = dict(title='Cumulative Miles: 2016-2019',
#               xaxis=dict(title='Day of Year'),
#               yaxis=dict(title='Miles'),
#               )

# data = [trace4, trace3, trace2, trace1, trace0]
# fig = dict(data=data, layout=layout)
# plotly.offline.iplot(fig)